In [7]:
from collections import Counter, defaultdict, deque
import copy
import math
import networkx as nx
import urllib.request

import matplotlib.pyplot as plt
%matplotlib inline 

In [8]:
def example_graph():
    """
    Create the example graph from class. Used for testing.
    Do not modify.
    """
    g = nx.Graph()
    g.add_edges_from([('A', 'B'), ('A', 'C'), ('B', 'C'), ('B', 'D'), ('D', 'E'), ('D', 'F'), ('D', 'G'), ('E', 'F'), ('G', 'F')])
    return g

In [9]:
graph= example_graph()


In [10]:
def bfs(graph, root, max_depth):
    """y
    Perform breadth-first search to compute the shortest paths from a root node to all
    other nodes in the graph. To reduce running time, the max_depth parameter ends
    the search after the specified depth.
    E.g., if max_depth=2, only paths of length 2 or less will be considered.
    This means that nodes greather than max_depth distance from the root will not
    appear in the result.
    You may use these two classes to help with this implementation:
      https://docs.python.org/3.5/library/collections.html#collections.defaultdict
      https://docs.python.org/3.5/library/collections.html#collections.deque
    Params:
      graph.......A networkx Graph
      root........The root node in the search graph (a string). We are computing
                  shortest paths from this node to all others.
      max_depth...An integer representing the maximum depth to search.
    Returns:
      node2distances...dict from each node to the length of the shortest path from
                       the root node
      node2num_paths...dict from each node to the number of shortest paths from the
                       root node that pass through this node.
      node2parents.....dict from each node to the list of its parents in the search
                       tree
    In the doctests below, we first try with max_depth=5, then max_depth=2.
    >>> node2distances, node2num_paths, node2parents = bfs(example_graph(), 'E', 5)
    >>> sorted(node2distances.items())
    [('A', 3), ('B', 2), ('C', 3), ('D', 1), ('E', 0), ('F', 1), ('G', 2)]
    >>> sorted(node2num_paths.items())
    [('A', 1), ('B', 1), ('C', 1), ('D', 1), ('E', 1), ('F', 1), ('G', 2)]
    >>> sorted((node, sorted(parents)) for node, parents in node2parents.items())
    [('A', ['B']), ('B', ['D']), ('C', ['B']), ('D', ['E']), ('F', ['E']), ('G', ['D', 'F'])]
    >>> node2distances, node2num_paths, node2parents = bfs(example_graph(), 'E', 2)
    >>> sorted(node2distances.items())
    [('B', 2), ('D', 1), ('E', 0), ('F', 1), ('G', 2)]
    >>> sorted(node2num_paths.items())
    [('B', 1), ('D', 1), ('E', 1), ('F', 1), ('G', 2)]
    >>> sorted((node, sorted(parents)) for node, parents in node2parents.items())
    [('B', ['D']), ('D', ['E']), ('F', ['E']), ('G', ['D', 'F'])]
    """
    distance = {}
    distance[root] = 0
    path = {}
    path[root] = 1
    parent ={}
    
    depth = 1
    
    visited_q = deque()
    visiting_q = deque()
    visiting_q.append(root)
    
    n  =defaultdict(list)
    for i in g:
         for j in g.edges(i):
            n[i].append(j[1])
    print (n)
    
    while depth <= max_depth:
        while len(visiting_q)>0:
            u = visiting_q.popleft()
            for i in n[u]:
                visited_q.append(i)
                if i not in distance.keys():
                    distance[i] = depth
                    parent[i]=[u]
                elif i in distance.keys() and distance[i] == depth:
                    parent[i].append(u) 
        depth = depth +1
        for m in visited_q:
            visiting_q.append(m)
        visited_q.clear()
    
    for i in parent:
        path[i] = len(parent[i])
        
    return distance,path,parent

In [11]:
node2distances, node2num_paths, node2parents = bfs(example_graph(), 'E',2)
print (sorted(node2distances.items()))
print (sorted(node2num_paths.items()))
print (sorted(node2parents.items()))

NameError: name 'g' is not defined

In [12]:
def bottom_up(root, node2distances, node2num_paths, node2parents):
    """
    Compute the final step of the Girvan-Newman algorithm.
    See p 352 From your text:
    https://github.com/iit-cs579/main/blob/master/read/lru-10.pdf
        The third and final step is to calculate for each edge e the sum
        over all nodes Y of the fraction of shortest paths from the root
        X to Y that go through e. This calculation involves computing this
        sum for both nodes and edges, from the bottom. Each node other
        than the root is given a credit of 1, representing the shortest
        path to that node. This credit may be divided among nodes and
        edges above, since there could be several different shortest paths
        to the node. The rules for the calculation are as follows: ...
    Params:
      root.............The root node in the search graph (a string). We are computing
                       shortest paths from this node to all others.
      node2distances...dict from each node to the length of the shortest path from
                       the root node
      node2num_paths...dict from each node to the number of shortest paths from the
                       root node that pass through this node.
      node2parents.....dict from each node to the list of its parents in the search
                       tree
    Returns:
      A dict mapping edges to credit value. Each key is a tuple of two strings
      representing an edge (e.g., ('A', 'B')). Make sure each of these tuples
      are sorted alphabetically (so, it's ('A', 'B'), not ('B', 'A')).
      Any edges excluded from the results in bfs should also be exluded here.
    >>> node2distances, node2num_paths, node2parents = bfs(example_graph(), 'E', 5)
    >>> result = bottom_up('E', node2distances, node2num_paths, node2parents)
    >>> sorted(result.items())
    [(('A', 'B'), 1.0), (('B', 'C'), 1.0), (('B', 'D'), 3.0), (('D', 'E'), 4.5), (('D', 'G'), 0.5), (('E', 'F'), 1.5), (('F', 'G'), 0.5)]
    """
    sortdis = sorted(node2distances.items(), key = lambda x: x[1], reverse = True)
    print ("step1", sortdis)
    node_credit = {}
    edge_credit = {}
    for i in sortdis:
        if i[0] != root:
            node_credit[i[0]] = 1
        else:
            node_credit[i[0]] = 0
    print ("step2",node_credit)
    print ("node2num_paths",node2num_paths)
    for i in sortdis:
        if i[0] != root:
            print ("asd",node2num_paths[i[0]])
            credit = node_credit[i[0]]/node2num_paths[i[0]]
            parent = node2parents[i[0]]
            for j in parent:
                node_credit[j] =node_credit[j] + credit
                print (j,node_credit)
                edge = sorted((i[0],j))
                edge_credit[tuple(edge)] = credit
                print ("edge_credit",edge_credit)
    return edge_credit

In [13]:
result = bottom_up('E', node2distances, node2num_paths, node2parents)
sorted(result.items())

NameError: name 'node2distances' is not defined

In [14]:
def approximate_betweenness(graph, max_depth):
    
    """
    Compute the approximate betweenness of each edge, using max_depth to reduce
    computation time in breadth-first search.

    You should call the bfs and bottom_up functions defined above for each node
    in the graph, and sum together the results. Be sure to divide by 2 at the
    end to get the final betweenness.

    Params:
      graph.......A networkx Graph
      max_depth...An integer representing the maximum depth to search.

    Returns:
      A dict mapping edges to betweenness. Each key is a tuple of two strings
      representing an edge (e.g., ('A', 'B')). Make sure each of these tuples
      are sorted alphabetically (so, it's ('A', 'B'), not ('B', 'A')).

    >>> sorted(approximate_betweenness(example_graph(), 2).items())
    [(('A', 'B'), 2.0), (('A', 'C'), 1.0), (('B', 'C'), 2.0), (('B', 'D'), 6.0), (('D', 'E'), 2.5), (('D', 'F'), 2.0), (('D', 'G'), 2.5), (('E', 'F'), 1.5), (('F', 'G'), 1.5)]
    
    """
    
    bet = {}
    for i in graph:
        node2distances1, node2num_paths, node2parents = bfs(graph, i,max_depth)
        result = bottom_up(i, node2distances, node2num_paths, node2parents)
        for i in result:
            if i not in bet.keys():
                bet[i] = result[i]/2
            else:
                bet[i] = bet[i] + (result[i]/2)
    return bet

In [15]:
sorted(approximate_betweenness(example_graph(), 2).items())

NameError: name 'g' is not defined

In [16]:
def is_approximation_always_right():
    """
    Look at the doctests for approximate betweenness. In this example, the
    edge with the highest betweenness was ('B', 'D') for both cases (when
    max_depth=5 and max_depth=2).

    Consider an arbitrary graph G. For all max_depth > 1, will it always be
    the case that the edge with the highest betweenness will be the same
    using either approximate_betweenness verses the exact computation?
    Answer this question below.

    In this function, you just need to return either the string 'yes' or 'no'
    (no need to do any actual computations here).
    >>> s = is_approximation_always_right()
    >>> type(s)
    <class 'str'>
    """
    ###TODO
    return yes
    pass

In [17]:
def partition_girvan_newman(graph, max_depth):
    """
    Use your approximate_betweenness implementation to partition a graph.
    Unlike in class, here you will not implement this recursively. Instead,
    just remove edges until more than one component is created, then return
    those components.
    That is, compute the approximate betweenness of all edges, and remove
    them until multiple comonents are created.

    You only need to compute the betweenness once.
    If there are ties in edge betweenness, break by edge name (e.g.,
    (('A', 'B'), 1.0) comes before (('B', 'C'), 1.0)).

    Note: the original graph variable should not be modified. Instead,
    make a copy of the original graph prior to removing edges.
    See the Graph.copy method https://networkx.github.io/documentation/development/reference/generated/networkx.Graph.copy.html
    Params:
      graph.......A networkx Graph
      max_depth...An integer representing the maximum depth to search.
    
    Returns:
      A list of networkx Graph objects, one per partition.
    
    >>> components = partition_girvan_newman(example_graph(), 5)
    >>> components = sorted(components, key=lambda x: sorted(x.nodes())[0])
    >>> sorted(components[0].nodes())
    ['A', 'B', 'C']
    >>> sorted(components[1].nodes())
    ['D', 'E', 'F', 'G']
    """
    ###TODO
    cg = graph.copy();
    
    print (approximate_betweenness(cg, 2).items())
    
    
    pass 

In [18]:
partition_girvan_newman(g, 5)

NameError: name 'g' is not defined

In [19]:
def evaluate(predicted_edges, graph):
    """
    Return the fraction of the predicted edges that exist in the graph.

    Args:
      predicted_edges...a list of edges (tuples) that are predicted to
                        exist in this graph
      graph.............a networkx Graph

    Returns:
      The fraction of edges in predicted_edges that exist in the graph.

    In this doctest, the edge ('D', 'E') appears in the example_graph,
    but ('D', 'A') does not, so 1/2 = 0.5

    >>> evaluate([('D', 'E'), ('D', 'A')], example_graph())
    0.5
    """
    ###TODO
    gp=[]
    node=[]
    
    for j in g.edges():
        
        if (j[1],i) not in gp:
            gp.append(j)
            print (j)"""
    
    for i in graph
   
    n= len(predicted_edges)
    
    for i in predicted_edges:
        for j in gp:
            if (i[0] == j[0] and i[1]==j[1]) or  (i[0] == j[1] and i[1]==j[0]):
                node.append(j)          

    print (len(node)/n)
        
            
    pass

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-19-3f0ffc1ef8da>, line 41)

In [20]:
evaluate([('F', 'E'),('D', 'E')], example_graph())

NameError: name 'evaluate' is not defined